In [22]:
import pandas as pd
import matplotlib.pyplot as plt

T = 3
face = 100
cpn = 0.06
freq = 1

rates = pd.read_excel('discount_curve_2025-02-13.xlsx')

def price_bond(face, cpn, freq, T, df: pd.Series):
    periods = int(T * freq)
    coupon_payment = face * cpn / freq
    cash_flows = [coupon_payment] * periods
    cash_flows[-1] += face  # Add face value to the last payment
    print(cash_flows)
    bond_price = 0
    for i in range( len(cash_flows)):
        discount_factor = df[df['ttm'] == (i + 1) / freq]['discount'].values[0]
        print(discount_factor)
        bond_price += cash_flows[i] * discount_factor
        
    return bond_price

In [27]:
vanilla_bond = price_bond(face, cpn, freq, T, rates[['ttm','discount']])

[6.0, 6.0, 106.0]
0.9584509053402022
0.9205151777624864
0.8840840562090114


In [28]:
from scipy.stats import norm
import numpy as np
Topt = 1.5
strike = 100
vol = 0.0268
forward_price = 103.31

def black_scholes_call(forward_price, strike, Topt, vol):
    
    d1 = (np.log(forward_price / strike) + 0.5 * vol**2 * Topt) / (vol * np.sqrt(Topt))
    d2 = d1 - vol * np.sqrt(Topt)
    
    call_value = np.exp(-0.05 * Topt) * (forward_price * norm.cdf(d1) - strike * norm.cdf(d2))
    return call_value

call_value = black_scholes_call(forward_price, strike, Topt, vol)
call_value

np.float64(3.332582017235195)

In [29]:
Call_price = vanilla_bond - call_value
Call_price

np.float64(101.65412443953615)

In [30]:
market_price = 3.5

def solve_for_volatility(market_price, forward_price, strike, Topt):
    from scipy.optimize import brentq
    
    def objective_function(vol):
        return black_scholes_call(forward_price, strike, Topt, vol) - market_price
    
    implied_vol = brentq(objective_function, 1e-6, 1.0)  # Search for volatility between 0 and 100%
    return implied_vol

implied_vol = solve_for_volatility(market_price, forward_price, strike, Topt)
implied_vol

0.032264321157916116